# Interactive spec creation

This notebook is intended to make it easier to generate ground truth for trips to put into the evaluation spec files. Putting in a bunch of trips into a spec file and trying to fill in all of them leads to wasted time, specially if we have a lot of relations, since we must query OSM over and over to pull the data to populate the coordinates. And then we have to visualize all the trips at once as part of the validation, so if we find any errors, we need to regenerate the coordinates for all trips.

This notebook allows us to interactively develop individual specs before copy-pasting them into the overall spec file.

### These initial cells are for setup and should not need to be changed

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

#### Let's see what we can do with the OSM API directly

In [ ]:
import autofill_eval_spec as aes

In [ ]:
import folium
import folium.features as fof
import branca.element as bre

In [ ]:
import polyline as pl

In [ ]:
def get_map_for_evaluation_trip(trip):
    curr_map = folium.Map()
    curr_start = lonlat_swap(trip["start_loc"]["coordinates"])
    curr_end = lonlat_swap(trip["end_loc"]["coordinates"])
    folium.Marker(curr_start, icon=folium.Icon(color="green"),
                  popup="Start: %s" % trip["start_loc"]["name"]).add_to(curr_map)
    folium.Marker(curr_end, icon=folium.Icon(color="red"),
                  popup="End: %s" % trip["end_loc"]["name"]).add_to(curr_map)
    # trips from relations won't have waypoints
    if "waypoint_coords" in trip:
        for i, wpc in enumerate(trip["waypoint_coords"]):
            folium.map.Marker(
                lonlat_swap(wpc), popup="%d" % i,
                icon=fof.DivIcon(class_name='leaflet-div-icon')).add_to(curr_map)
    print("Found %d coordinates for the route" % (len(trip["route_coords"])))
    latlng_route_coords = [lonlat_swap(rc) for rc in trip["route_coords"]]
    folium.PolyLine(latlng_route_coords,
                    popup="%s: %s" % (trip["mode"], trip["name"])).add_to(curr_map)
    for i, c in enumerate(latlng_route_coords):
        folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(curr_map)
    curr_map.fit_bounds([curr_start, curr_end])    
    return curr_map

In [ ]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

# This is the important part

Instructions:
- Edit the cell below to include the spec that you are experimenting with
- Run the cells below that to fill it in and display it
- Tweak the various fill/population methods until the trip looks right
- Copy/paste the leg spec into your spec file
- Rinse and repeat

In [ ]:
curr_leg_spec = \
        {
            "id": "unexpected_bike_to_bus",
            "name": "Bike to bus when stop changed due to Thursday night live",
            "mode": "BICYCLING",
            "start_loc": {
                "osm_id": 5020708467,
                "name": "Mountain View City Hall"
            },
            "end_loc": {
                "osm_id": 726963272,
                "name": "El Camino and Miramonte (alternate stop)"                
            },
            "waypoint_coords": [
          [
            -122.0817732810974,
            37.38932665304395
          ],
          [
            -122.08233386278151,
            37.38833995211305
          ],
          [
            -122.08664685487746,
            37.39046251514165
          ],
          [
            -122.08813816308974,
            37.38838683668442
          ],
          [
            -122.08862364292143,
            37.38818438037076
          ]
            ]
        }

In [ ]:
aes.validate_and_fill_leg(curr_leg_spec)

In [ ]:
get_map_for_evaluation_trip(curr_leg_spec)